## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

#### What's the `version.build_hash` value?
- "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"



### Getting the Data

In [9]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

In [12]:
from elasticsearch import Elasticsearch

In [13]:
es_client = Elasticsearch('http://localhost:9200')

In [15]:
index_Settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-question-all"

es_client.indices.create(index=index_name, body=index_Settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-question-all'})

### Which function do you use for adding your data to elastic?
- Index Function

In [16]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [17]:
query = 'How do I execute a command in a running docker container?'

### Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

### What's the score for the top ranking result?
 #### Score: 84.050095

In [21]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], 
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Source: {hit['_source']}")

Score: 84.050095, Source: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
Score: 75.54128, Source: {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword

## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

How do I debug a docker container?
How do I copy files from a different folder into docker container’s working directory?
How do Lambda container images work?
How can I annotate a graph?

In [49]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], 
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [51]:
elastic_search('How do I debug a docker container?')


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [52]:
elastic_search('How do I copy files from a different folder into docker container’s working directory?')


[{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'questi

In [53]:
elastic_search('How do Lambda container images work?')

[{'text': 'I wanted to understand how lambda container images work in depth and how lambda functions are initialized, for this reason, I found the following documentation\nhttps://docs.aws.amazon.com/lambda/latest/dg/images-create.html\nhttps://docs.aws.amazon.com/lambda/latest/dg/runtimes-api.html\nAdded by Alejandro aponte',
  'section': '9. Serverless Deep Learning',
  'question': 'How do Lambda container images work?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Tim from BentoML has prepared a dedicated video tutorial wrt this use case here:\nhttps://www.youtube.com/watch?v=7gI1UH31xb4&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR&index=97\nKonrad Muehlberg',
  'section': 'Miscellaneous',
  'question': 'How to pass BentoML content / docker container to Amazon Lambda',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the

In [54]:
elastic_search('How can I annotate a graph?')

[{'text': "Matplotlib has a cool method to annotate where you could provide an X,Y point and annotate with an arrow and text. For example this will show an arrow pointing to the x,y point optimal threshold.\nplt.annotate(f'Optimal Threshold: {optimal_threshold:.2f}\\nOptimal F1 Score: {optimal_f1_score:.2f}',\nxy=(optimal_threshold, optimal_f1_score),\nxytext=(0.3, 0.5),\ntextcoords='axes fraction',\narrowprops=dict(facecolor='black', shrink=0.05))\nQuinn Avila",
  'section': '4. Evaluation Metrics for Classification',
  'question': 'How can I annotate a graph?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'I like this visual implementation of features importance in scikit-learn library:\nhttps://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html\nIt actually adds std.errors to features importance -> so that you can trace stability of features (important for a model’s explainability) over the different params of the model.\nIvan Brigida',
  'section': '

## Q5. Building a prompt

In [57]:
query = "How do I execute a command in a running docker container?"
search_result = elastic_search(query)


In [72]:
def build_prompt(query, search_results):
    prompt_template = """
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    prompt_length = len(prompt)

    print("Prompt len is:", prompt_length)
    return prompt
    
    
    


In [85]:
prompt = build_prompt(query, search_result)
print(prompt)

Prompt len is: 1462
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

question: How do I copy files from my local machine to docker container?
answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory conta

## Q6. Tokens

In [75]:
import tiktoken

In [76]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [88]:
token = encoding.encode(prompt)
num_tokens = len(token)
print("Prompt number of Tokens", num_tokens)

print("Decoded Token:", encoding.decode_single_token_bytes(63842))

Prompt number of Tokens 316
Decoded Token: b"You're"


## Bonus: generating the answer (ungraded)

### Let's send the prompt to OpenAI. What's the response?

In [92]:
from openai import OpenAI
client = OpenAI()

In [93]:
response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content":prompt}]
)

In [95]:
print(response.choices[0].message.content)

To execute a command in a running docker container, follow these steps:

1. Find the container ID by listing the running containers using:
   ```
   docker ps 
   ```
2. Once you have the container ID, execute a command in the container using:
   ```
   docker exec -it <container-id> bash
   ```


### on June 25, the prices for gpt4o are:

Input: $0.005 / 1K tokens

Output: $0.015 / 1K tokens